In [ ]:
# Solution 2.3):

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

torch.cuda.empty_cache()
# Define the transformations to be applied to the images

# ConvModule: a convolutional module in the above picture, consists a 2d convolutional layer, a 2d batchnorm layer, and a ReLU activation.
class ConvModule(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride, padding='same'):
        super(ConvModule, self).__init__()
        self.conv2d = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv2d(x)
        x = self.batchnorm(x)
        x = self.relu(x)

        return x

# InceptionModule: a inception module in the above picture, consists a convolution module with 1x1 filter, 
# a convolution module with 3x3 filter, then concatenate these two outputs.
class InceptionModule(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3):
        super(InceptionModule, self).__init__()

        self.conv1x1 = ConvModule(in_channels, ch1x1, (1, 1), 1)
        self.conv3x3 = ConvModule(in_channels, ch3x3, (3, 3), 1)

    def forward(self, x):
        out1 = self.conv1x1(x)
        out2 = self.conv3x3(x)
        x = torch.cat((out1, out2), 1)
        return x

# DownsampleModule: a downsample module in the above picture, consists a convolution module with 3x3 filter,
# a 2d maxpool layer, then concatenate these two outputs.
class DownsampleModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownsampleModule, self).__init__()

        self.conv3x3 = ConvModule(in_channels, out_channels, (3, 3), (2, 2), padding='valid')
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        out1 = self.conv3x3(x)
        out2 = self.maxpool(x)

        #return out1
        x = torch.cat((out1, out2), 1)

        return x
    

# MiniGoogLeNet: the MiniGoogLeNet model. Input: input_channels * 32 * 32. 
# When input_channels is 1, the input is a grayscale image. When input_channels is 3, the input is a RGB image.
# Output: a tensor with the shape of [-1, classes], where classes it the number of classes.

class MiniGoogLeNet(nn.Module):
    def __init__(self, classes, input_channels):
        super(MiniGoogLeNet, self).__init__()

        self.conv1 = ConvModule(input_channels, 96, kernel_size=(3, 3), stride=1) # input_channel is 3 if you want to deal with RGB image, 1 for grey scale image
        self.inception1 = InceptionModule(96, 32, 32)
        self.inception2 = InceptionModule(32+32, 32, 48)
        self.downsample1 = DownsampleModule(32+48, 80)

        self.inception3 = InceptionModule(80+80, 112, 48)
        self.inception4 = InceptionModule(112+48, 96, 64)
        self.inception5 = InceptionModule(96+64, 80, 80)
        self.inception6 = InceptionModule(80+80, 48, 96)
        self.downsample2 = DownsampleModule(48+96, 96)

        self.inception7 = InceptionModule(96+96, 176, 160)
        self.inception8 = InceptionModule(176+160, 176, 160)
        self.avgpool2d = nn.AvgPool2d(kernel_size=7)
        self.dropout = nn.Dropout2d(0.5)

        self.fc = nn.Linear(240, classes)
        #self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.conv1(x)
        #print(x.shape)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.downsample1(x)

        x = self.inception3(x)
        x = self.inception4(x)
        x = self.inception5(x)
        x = self.inception6(x)
        x = self.downsample2(x)

        x = self.avgpool2d(x)
        x = self.dropout(x)

        x = torch.flatten(x, 1)
        x = self.fc(x)
        #x = self.softmax(x), no need for softmax because PyTorch Cross Entropy Loss implemented softmax

        return x

lr = 1e-09
BatchSize = [32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]
# BatchSize = [32, 64, 128, 256, 512, 1024, 2048]
batch_multiplier = 16

# Train the model for 5 epochs for each learning rate candidate and store the results
losses = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for Batch in BatchSize:
    torch.cuda.empty_cache()
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Resize((32,32))
    ])
    # Load the fashionMNIST dataset
    train_set = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=Batch, shuffle=True)
    print(f'Training with Batch Size {Batch}')
    model = MiniGoogLeNet(10, 1).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    for epoch in range(5):
        count = 0
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            if(count == 0):
                optimizer.step()
                optimizer.zero_grad()
                count = batch_multiplier
            outputs = model(inputs)
            loss = criterion(outputs, labels) / batch_multiplier
            loss.backward()
            count -= 1
            running_loss += loss.item()
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1} loss: {epoch_loss:.4f}')
        torch.cuda.empty_cache()
    losses.append(epoch_loss)

# Plot the losses as a function of the learning rate
plt.plot(BatchSize, losses)
plt.xscale('log')
plt.xlabel('Batch Size')
plt.ylabel('Training Loss')
plt.show()